In [ ]:
import operator
from typing import Annotated, List, TypedDict, Union

from langchain_openai import ChatOpenAI
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import create_react_agent


# 1. Define the Shared State
class AgentState(TypedDict):
    # The 'messages' list will store the conversation history
    # operator.add ensures new messages are appended rather than overwritten
    messages: Annotated[List[BaseMessage], operator.add]
    next: str  # Tracks which agent should go next


# 2. Initialize the LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0)


# 3. Create the Specialist Agents
# Research Agent: Uses a dummy tool to "search"
def dummy_search(query: str):
    return f"Search results for '{query}': LangGraph is the best for multi-agent workflows."


research_agent = create_react_agent(llm, tools=[dummy_search])
writer_agent = create_react_agent(llm, tools=[])

# 4. Define the Supervisor (The Brain)
# This node decides who speaks next or if we should STOP.
members = ["Researcher", "Writer"]
system_prompt = (
    "You are a supervisor tasked with managing a conversation between"
    f" the following workers: {members}. Given the user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results. When finished, respond with FINISH."
)


def supervisor_node(state: AgentState):
    messages = [{"role": "system", "content": system_prompt}] + state["messages"]
    # We use structured output to ensure the LLM only picks a member or FINISH
    response = llm.with_structured_output(TypedDict("Route", {"next": str})).invoke(
        messages
    )
    return {"next": response["next"]}


# 5. Build the Graph
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("supervisor", supervisor_node)
workflow.add_node(
    "Researcher",
    lambda state: {"messages": [research_agent.invoke(state["messages"])[-1]]},
)
workflow.add_node(
    "Writer", lambda state: {"messages": [writer_agent.invoke(state["messages"])[-1]]}
)

# Define edges
for member in members:
    # After a worker finishes, they always go back to the supervisor
    workflow.add_edge(member, "supervisor")

# The supervisor uses conditional edges to route the flow
conditional_map = {m: m for m in members}
conditional_map["FINISH"] = END

workflow.add_conditional_edges(
    "supervisor", lambda state: state["next"], conditional_map
)
workflow.add_edge(START, "supervisor")

# 6. Compile and Run
app = workflow.compile()

for s in app.stream(
    {
        "messages": [
            HumanMessage(content="Research LangGraph and write a 1-sentence summary.")
        ]
    }
):
    if "__metadata__" not in str(s):
        print(s)
        print("-" * 50)